In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
model.eval()

Using cache found in /home/suchira/.cache/torch/hub/pytorch_vision_v0.10.0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [2]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
img_path = 'inputs/dog.jpg'

In [4]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(img_path)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([-2.3523e+00, -2.5592e+00, -1.4565e+00, -2.9491e+00, -2.5645e+00,
        -1.9039e+00, -2.7425e+00,  2.0768e+00,  4.2541e+00, -1.4167e+00,
        -3.7602e+00, -2.4736e+00, -3.2579e+00, -2.0015e+00, -2.4200e+00,
        -2.5387e+00, -1.0398e+00, -5.4853e-01,  2.9849e-01, -2.4379e+00,
        -1.9159e+00, -1.5719e+00, -6.8891e-01,  9.3929e-01, -2.2620e+00,
        -2.4495e+00, -3.1072e+00, -1.4747e+00, -2.2212e+00, -8.2793e-01,
        -3.6602e+00, -2.4029e+00, -3.2572e+00, -4.3836e+00, -3.3341e+00,
        -3.3730e+00, -2.2817e+00, -2.4155e+00, -4.3661e+00, -2.8331e+00,
        -1.7350e+00, -3.4137e+00, -4.6674e+00, -4.0638e+00, -3.6041e+00,
        -2.5002e+00, -3.9387e-02, -3.7297e+00, -3.4307e+00, -2.5803e+00,
        -1.2164e+00, -3.4983e+00, -1.5988e+00, -2.9984e+00, -2.7314e+00,
        -2.1666e+00, -3.0543e+00, -3.4226e+00, -2.9685e+00, -2.5581e+00,
        -1.6249e+00, -4.8276e+00, -4.6968e+00, -2.9227e+00, -2.8231e+00,
        -4.0451e+00, -3.6469e+00, -3.3718e+00, -4.5

In [5]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.8397048711776733
Pomeranian 0.032958149909973145
Eskimo dog 0.016814857721328735
white wolf 0.015938688069581985
Great Pyrenees 0.012166241183876991


In [6]:
prediction = torch.topk(probabilities, 5)
print(f"Prediction = {categories[top5_catid[0]]}")

Prediction = Samoyed
